In [ ]:
import numpy as np
import training_data_generation as tdg
num_examples = 10
run_dir = "Data"

# Generate and Save Training Data

In [ ]:
dataset_params, sys_params, facility_spec = tdg.main((_, run_dir, num_examples))

# Diagnostic

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
# sys_params["run_clean"] must be false before run for diagnostic

In [ ]:
sys_params = tdg.define_system_params("Data")
sys_params["root_dir"] = "Data"
sys_params["run_gen_deck"] = False
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
iex = 0

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
theta_slice = slice(0,29,4)
phi_slice   = slice(1,30,4)
power_slice = slice(3,32,4)

cone_theta_offset = dataset_params["sim_params"][theta_slice,iex]
cone_phi_offset = dataset_params["sim_params"][phi_slice,iex]
cone_powers = dataset_params["sim_params"][power_slice,iex]

beams_prev = 0
beams_tot = 0
total_power = 0
for icone in range(facility_spec['num_cones']):
    beams_per_cone = facility_spec['beams_per_cone'][icone]
    beams_tot += beams_per_cone
    total_power += cone_powers[icone] * beams_per_cone
    beams_prev += beams_per_cone
mean_power_fraction = total_power / facility_spec['nbeams']

intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)

hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(dataset_params["theta_pointings"][:,iex], dataset_params["phi_pointings"][:,iex])
plt.savefig(sys_params["figure_location"]+'/intensity_mollweide.png', dpi=300, bbox_inches='tight')

In [ ]:
hp.mollview(intensity_map_rms_spatial, unit="Deviation from mean (%)",flip="geo")
plt.savefig(sys_params["figure_location"]+'/deviation_from_mean_mollweide.png', dpi=300, bbox_inches='tight')
hp.graticule()

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = 20

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes LLE")
plt.xlabel("l mode")
plt.ylabel(r"rms amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"rms amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes.png", dpi=300, bbox_inches='tight')

In [ ]:
intensity_map_normalized, avg_power = uim.imap_norm(intensity_map)
hp.mollview(intensity_map_normalized, unit="Deviation from Mean", flip="geo")

X_train = uim.imap2xtrain(intensity_map_normalized, LMAX, avg_power)
intensity_map_normalized2 = uim.xtrain2imap(X_train, LMAX, dataset_params["imap_nside"], avg_power)
hp.mollview(intensity_map_normalized2, unit="Deviation from Mean",flip="geo")

# Combine Datasets

In [ ]:
from netCDF4 import Dataset
from os import path
import os
import numpy as np

In [ ]:
dataset1 = "Data1"
dataset2 = "Data2"

new_file = "Data3"

filename_trainingdata = dataset1 + "/" + sys_params["trainingdata_filename"] 
training_data = Dataset(filename_trainingdata)
X_1 = training_data.variables["X_train"][:]
Y_1 = training_data.variables["Y_train"][:]
avg_powers_1 = training_data.variables["avg_powers"][:]
training_data.close()

filename_trainingdata = dataset2 + "/" + sys_params["trainingdata_filename"] 
training_data = Dataset(filename_trainingdata)
X_2 = training_data.variables["X_train"][:]
Y_2 = training_data.variables["Y_train"][:]
avg_powers_2 = training_data.variables["avg_powers"][:]
training_data.close()

X_train = np.hstack((X_1, X_2))
Y_train = np.hstack((Y_1, Y_2))
avg_powers = np.hstack((avg_powers_1, avg_powers_2))
print(np.shape(X_train), np.shape(Y_train), np.shape(avg_powers))

filename_trainingdata = new_file + "/" + sys_params["trainingdata_filename"] 
nrw.save_training_data(X_train, Y_train, avg_powers, filename_trainingdata)

# Check Shuffle Maintained labelling

In [ ]:
import healpy as hp
import numpy as np
import utils_intensity_map as uim
import neural_network_generation as nng
num_epochs = 10
num_nn = -1
nn_params, nn_dataset, sys_params, nn_hyperparams = nng.main((_, run_dir, num_epochs, num_nn))
X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data(nn_params, sys_params)

In [ ]:
if nn_params["run_shuffle"]:
    ind = 0
    ind_first = np.where(nn_dataset["index_shuf"] == ind)
    ind_first = np.squeeze(ind_first[0])

    print(Y_all[:,ind])
    intensity_map_normalized2 = uim.xtrain2imap(X_all[:,ind], dataset_params["LMAX"], dataset_params["imap_nside"], avg_powers_all[ind])
    hp.mollview(intensity_map_normalized2, unit="Deviation from Mean",flip="geo")

    if ind_first >= nn_params["test_size"]:
        i = ind_first - nn_params["test_size"]
        print(nn_dataset["Y_train"][i,:])
        cdata = nn_dataset["X_train"][i,:] * nn_dataset["sigma"] + nn_dataset["mu"]
        apwrs = nn_dataset["train_avg_powers"][i]
    else:
        i = ind_first
        print(nn_dataset["Y_test"][i,:])
        cdata = nn_dataset["X_test"][i,:] * nn_dataset["sigma"] + nn_dataset["mu"]
        apwrs = nn_dataset["test_avg_powers"][i]
    intensity_map_normalized2 = uim.xtrain2imap(cdata, dataset_params["LMAX"], dataset_params["imap_nside"], apwrs)
    hp.mollview(intensity_map_normalized2, unit="Deviation from Mean",flip="geo")

# Neural Network

In [ ]:
import neural_network_generation as nng
import matplotlib.pyplot as plt
num_epochs = 100
num_nn = 1

In [ ]:
nn_params, nn_dataset, sys_params, nn_hyperparams = nng.main((_, run_dir, num_epochs, num_nn))

In [ ]:
fig = plt.figure()
ax1 = plt.axes()
ax1.plot(nn_hyperparams["initialize_seed"], nn_hyperparams["test_acc"], "x")

## Run Prediction Through Ifriit

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
%matplotlib inline
num_epochs = 10
num_nn = -1
#nn_params, nn_dataset, sys_params, nn_hyperparams = nng.main((_, run_dir, num_epochs, num_nn))

In [ ]:
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)

# choose neural network
nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))

# choose test data set
sys_params = tdg.define_system_params("Data")
nn_params = nng.define_nn_params(num_nn)
X_all, Y_all, avg_powers_all, nn_params = nng.import_training_data(nn_params, sys_params)
nn_params["test_fraction"] = 0.0
nn_params["test_size"] = 0
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
nn_dataset = nng.normalise(nn_dataset)
dataset_params["Y_train"] = nn_dataset["Y_train"].T
y_true = dataset_params["Y_train"]

# apply neural net to test set
sys_params = tdg.define_system_params("Data_test_pred_1")
dataset_params, facility_spec = tdg.define_dataset_params(nn_params["num_examples"])
y_pred = tfnn.apply_network(nn_dataset["X_train"], nn_weights)
dataset_params["Y_train"] = y_pred

print("The absolute error is: ", np.mean(np.abs(y_true - y_pred)))
print("The percentage relative error is: ", np.mean((np.abs(y_true - y_pred)) / y_true) * 100, "%")

In [ ]:
# Create new run files
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
sys_params = tdg.define_system_params("Data_test_pred_1")
sys_params["root_dir"] = "Data_test_pred_1"
sys_params["run_gen_deck"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
# plot comparison
iex = 0

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
theta_slice = slice(0,29,4)
phi_slice   = slice(1,30,4)
power_slice = slice(3,32,4)

cone_theta_offset = dataset_params["sim_params"][theta_slice,iex]
cone_phi_offset = dataset_params["sim_params"][phi_slice,iex]
cone_powers = dataset_params["sim_params"][power_slice,iex]

beams_prev = 0
beams_tot = 0
total_power = 0
for icone in range(facility_spec['num_cones']):
    beams_per_cone = facility_spec['beams_per_cone'][icone]
    beams_tot += beams_per_cone
    total_power += cone_powers[icone] * beams_per_cone
    beams_prev += beams_per_cone
mean_power_fraction = total_power / facility_spec['nbeams']

intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)

hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(dataset_params["theta_pointings"][:,iex], dataset_params["phi_pointings"][:,iex])

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = 20

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes LLE")
plt.xlabel("l mode")
plt.ylabel(r"rms amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"rms amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes.png", dpi=300, bbox_inches='tight')

## Compare NN

In [ ]:
import netcdf_read_write as nrw

In [ ]:
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

fig = plt.figure()
ax1 = plt.axes()
ax1.plot(nn_hyperparams["initialize_seed"], nn_hyperparams["test_acc"], "x")

## Compare NN accuracy from weights

In [ ]:
import tf_neural_network as tfnn
import netcdf_read_write as nrw

In [ ]:
num_nn = 1
initialize_seed = np.arange(num_nn)
test_acc = np.zeros(num_nn)
input_size = dataset_params["num_coeff"] * 2

for inn in range(num_nn):
    nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
    y_pred = tfnn.apply_network(x_test, nn_weights)
    test_acc[inn] = np.mean(np.abs(nn_dataset["Y_train"] - y_pred.T))

fig = plt.figure()
ax1 = plt.axes()

ax1.plot(initialize_seed, test_acc)
plt.savefig(sys_params["figure_location"]+'/Compare_NN.png', dpi=300, bbox_inches='tight')

## Open NN Weights

In [ ]:
parameters = nrw.read_nn_weights(nn_params["filename_nn_weights"])

## Restart Training

In [ ]:
"""
start_epoch = num_epochs
finish_epoch = 1000 # needs to be bigger than start_epoch

parameters, costs2, train_acc2, test_acc2, epochs2 = tfnn.model_wrapper(X_train, Y_train, X_test, Y_test, learning_rate = lr, num_epochs = finish_epoch, start_epoch = start_epoch, nn_weights = parameters)
filename_nn_weights = dir_nn_weights + "/NN" + str(0)
nrw.save_nn_weights(parameters, filename_nn_weights)

costs = np.append(costs, costs2)
train_acc = np.append(train_acc, train_acc2)
test_acc = np.append(test_acc, test_acc2)
epochs = np.append(epochs, epochs2)
"""

In [ ]:
#nnp.plotting(epochs, costs, train_acc, test_acc, lr, sys_params["figure_location"])

## Apply NN

In [ ]:
"""
ie = 0

x_test = np.reshape(X_test[ie,:], (-1,input_size))
y_true = np.squeeze(Y_test[ie,:])

y_pred = tfnn.apply_network(x_test, parameters)
y_pred = np.squeeze(y_pred)
print(y_pred)
print(y_true)
print((np.abs(y_true - y_pred)) / y_true * 100)
print(np.mean((np.abs(y_true - y_pred)) / y_true * 100))
"""

In [ ]:
"""
x_test = X_test
y_true = np.squeeze(Y_test[:,:])
y_true = Y_test

y_pred = tfnn.apply_network(x_test, parameters)
y_pred = y_pred.T
print(np.shape(y_true), np.shape(x_test), np.shape(y_pred))
y_pred = np.squeeze(y_pred)
print(np.mean(y_true), np.mean(y_pred))
print(np.mean(np.abs(y_true - y_pred), axis=0))
print(np.mean(np.abs(y_true - y_pred)/y_true*100.0, axis=0))
plt.hist(y_true.flatten())
plt.hist(y_pred.flatten())
plt.savefig(sys_params["figure_location"]+'/label_distribution_blue_is_true.png', dpi=300, bbox_inches='tight')
"""